# Time Series Animation for Remote Sensing Data

This notebook demonstrates how to create interactive time series animations for remote sensing data using `keplergl` in Python. It visualizes temporal changes in a stack of raster data (e.g., NDVI from Sentinel-2 imagery) over time, overlaid with vector data (e.g., AOI boundaries). This is useful for monitoring changes like vegetation growth, land cover dynamics, or urban expansion.

## Prerequisites
- Install required libraries: `keplergl`, `rasterio`, `geopandas`, `numpy`, `matplotlib`, `imageio` (listed in `requirements.txt`).
- A stack of preprocessed GeoTIFF files representing a time series (e.g., from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- A GeoJSON or shapefile defining the area of interest (AOI) (e.g., `aoi.geojson`).
- Replace file paths with your own data.

## Learning Objectives
- Load and preprocess a time series of raster data.
- Compute a vegetation index (e.g., NDVI) for each time step.
- Create an interactive time series animation with Kepler.gl.
- Save the animation as an HTML file for sharing.

In [ ]:
# Import required libraries
from keplergl import KeplerGl
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import imageio
from rasterio.warp import transform_bounds
import glob
import os
from datetime import datetime

## Step 1: Load Time Series Rasters and AOI

Load a series of preprocessed GeoTIFF files and the AOI vector data.

In [ ]:
# Define file paths
raster_dir = 'remote_sensing_data/time_series/'  # Directory with time series GeoTIFFs
aoi_path = 'aoi.geojson'                        # Replace with your AOI file
raster_files = sorted(glob.glob(os.path.join(raster_dir, '*.tif')))  # Replace with your file pattern

# Load rasters and extract dates
raster_stack = []
dates = []
for file in raster_files:
    with rasterio.open(file) as src:
        raster_stack.append(src.read(masked=True))  # Shape: (bands, height, width)
        profile = src.profile
        raster_crs = src.crs
    # Extract date from filename (assumes format like 'sentinel_2023_01_01.tif')
    date_str = os.path.basename(file).split('_')[1]  # Adjust based on your naming convention
    dates.append(datetime.strptime(date_str, '%Y_%m_%d'))

# Convert to numpy array
raster_stack = np.stack(raster_stack, axis=0)  # Shape: (time, bands, height, width)

# Load AOI
aoi_gdf = gpd.read_file(aoi_path)
if aoi_gdf.crs != raster_crs:
    aoi_gdf = aoi_gdf.to_crs(raster_crs)

# Print basic information
print(f'Time series shape: {raster_stack.shape}')
print(f'Dates: {dates}')
print(f'Raster CRS: {raster_crs}')
print(f'AOI CRS: {aoi_gdf.crs}')

## Step 2: Compute NDVI for Each Time Step

Calculate the Normalized Difference Vegetation Index (NDVI) for each raster in the time series.

In [ ]:
# Assume Sentinel-2 bands: B4 (Red, index 0), B8 (NIR, index 3)
red_band_idx = 0
nir_band_idx = 3

# Compute NDVI: (NIR - Red) / (NIR + Red)
ndvi_stack = np.zeros((len(raster_stack), raster_stack.shape[2], raster_stack.shape[3]), dtype=np.float32)
for t in range(len(raster_stack)):
    red = raster_stack[t, red_band_idx].astype(float)
    nir = raster_stack[t, nir_band_idx].astype(float)
    ndvi_stack[t] = np.where((nir + red) != 0, (nir - red) / (nir + red), np.nan)

# Visualize NDVI for the first time step
plt.figure(figsize=(8, 8))
plt.imshow(ndvi_stack[0], cmap='RdYlGn', vmin=-1, vmax=1)
plt.colorbar(label='NDVI')
plt.title(f'NDVI - {dates[0].strftime("%Y-%m-%d")}')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 3: Prepare NDVI Rasters for Kepler.gl

Normalize NDVI data and save each time step as a PNG for Kepler.gl animation.

In [ ]:
# Create temporary directory for PNGs
temp_dir = 'temp_ndvi_pngs/'
os.makedirs(temp_dir, exist_ok=True)

# Normalize NDVI and save as PNGs
png_files = []
with rasterio.open(raster_files[0]) as src:
    bounds = src.bounds
    raster_crs = src.crs
bounds_latlon = transform_bounds(raster_crs, 'EPSG:4326', *bounds)

for t, date in enumerate(dates):
    ndvi_normalized = (ndvi_stack[t] + 1) / 2  # Scale NDVI (-1,1) to (0,1)
    ndvi_normalized = np.clip(ndvi_normalized, 0, 1)
    png_path = os.path.join(temp_dir, f'ndvi_{date.strftime("%Y%m%d")}.png')
    imageio.imwrite(png_path, (ndvi_normalized * 255).astype(np.uint8))
    png_files.append(png_path)

print(f'Saved {len(png_files)} NDVI PNGs to: {temp_dir}')

## Step 4: Create Kepler.gl Time Series Animation

Initialize a Kepler.gl map and add NDVI time series and AOI as layers.

In [ ]:
# Initialize Kepler.gl map
map_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': (bounds_latlon[1] + bounds_latlon[3]) / 2,
            'longitude': (bounds_latlon[0] + bounds_latlon[2]) / 2,
            'zoom': 10
        },
        'mapStyle': {
            'styleType': 'dark'
        }
    }
}
m = KeplerGl(height=600, config=map_config)

# Add NDVI time series as image layers with timestamps
for png_path, date in zip(png_files, dates):
    m.add_data(
        data={
            'type': 'image',
            'url': png_path,
            'bounds': [bounds_latlon[0], bounds_latlon[1], bounds_latlon[2], bounds_latlon[3]],
            'timestamp': date.strftime('%Y-%m-%d')
        },
        name=f'NDVI_{date.strftime("%Y%m%d")}'
    )

# Add AOI as vector layer
temp_geojson = 'temp_aoi.geojson'
aoi_gdf.to_crs('EPSG:4326').to_file(temp_geojson, driver='GeoJSON')
with open(temp_geojson, 'r') as f:
    m.add_data(data=f.read(), name='AOI')

# Configure time series settings
m.config['config']['visState'] = {
    'layers': [
        {
            'type': 'grid',
            'config': {
                'dataId': f'NDVI_{dates[0].strftime("%Y%m%d")}',
                'visualChannels': {'colorField': None, 'colorScale': 'quantile'}
            }
        },
        {
            'type': 'geojson',
            'config': {
                'dataId': 'AOI',
                'visualChannels': {'colorField': None, 'color': [255, 0, 0], 'strokeColor': [255, 0, 0]}
            }
        }
    ],
    'filters': [
        {
            'dataId': [f'NDVI_{date.strftime("%Y%m%d")}' for date in dates],
            'name': 'timestamp',
            'type': 'timeRange',
            'value': [int(dates[0].timestamp() * 1000), int(dates[-1].timestamp() * 1000)]
        }
    ]
}

# Display map
m

## Step 5: Save the Interactive Animation

Save the Kepler.gl map as an HTML file for sharing.

In [ ]:
# Save map to HTML
output_map_path = 'time_series_animation.html'
m.save_to_html(file_name=output_map_path)

print(f'Interactive animation saved to: {output_map_path}')

# Clean up temporary files
for png in png_files:
    os.remove(png)
os.rmdir(temp_dir)
os.remove(temp_geojson)

## Step 6: Static Visualization for Reference

Create a static plot of NDVI for a middle time step with AOI overlay.

In [ ]:
# Select middle time step
mid_idx = len(dates) // 2

# Plot NDVI with AOI
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(ndvi_stack[mid_idx], cmap='RdYlGn', vmin=-1, vmax=1)
aoi_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
plt.colorbar(label='NDVI')
plt.title(f'NDVI with AOI - {dates[mid_idx].strftime("%Y-%m-%d")}')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Next Steps

- Replace `remote_sensing_data/time_series/*.tif` with your own time series GeoTIFFs (e.g., from `21_download_data.ipynb`).
- Update `aoi.geojson` with your area of interest file.
- Adjust the date parsing logic in Step 1 to match your file naming convention.
- Customize the Kepler.gl animation by adding other indices (e.g., NDWI) or tweaking layer styles in the Kepler.gl interface.
- Proceed to analysis notebooks like `17_time_series_analysis.ipynb` for quantitative analysis or `15_unet_segmentation.ipynb` for segmentation of time series data.

## Notes
- Ensure all rasters in the time series have the same CRS, resolution, and dimensions (see `24_advanced_preprocessing.ipynb`).
- Large raster stacks may require downsampling to avoid memory issues in Kepler.gl.
- Use the Kepler.gl interface to adjust animation speed, color scales, or layer visibility.
- See `docs/installation.md` for troubleshooting library installation.